In [1]:
from itertools import islice
import numpy as np

from model import MidiModel
from input import note_batch_generator, bits_to_string, read_songs
from midi import create_and_save_midi_file
from random import shuffle

In [2]:
data_path = "out-off12-fpiano.txt"
time_len = 16
batch_len = 64

In [3]:
songs = list(read_songs(data_path))
shuffle(songs)

In [4]:
total_song_count = len(songs)
train_song_count = int(total_song_count * 0.8)
test_song_count = total_song_count - train_song_count
train_songs = songs[:train_song_count]
test_songs = songs[train_song_count:]

In [6]:
midi_model = MidiModel(0.5, time_len, batch_len)    
test_result = midi_model.train(500, train_songs, test_songs)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (64, 16, 64)              8256      
_________________________________________________________________
lstm_1 (LSTM)                (64, 128)                 98816     
_________________________________________________________________
dense_3 (Dense)              (64, 128)                 16512     
Total params: 123,584
Trainable params: 123,584
Non-trainable params: 0
_________________________________________________________________
Epoch: 1/500 Loss: 0.0455 LR: 1.000e-03 TestLoss: 0.0478 L+: 0 L-: 1
Epoch: 2/500 Loss: 0.0402 LR: 1.000e-03 TestLoss: 0.0442 L+: 0 L-: 2
Epoch: 3/500 Loss: 0.0361 LR: 1.000e-03 TestLoss: 0.0358 L+: 0 L-: 3
Epoch: 4/500 Loss: 0.0304 LR: 1.000e-03 TestLoss: 0.0309 L+: 0 L-: 4
Epoch: 5/500 Loss: 0.0269 LR: 1.000e-03 TestLoss: 0.0281 L+: 0 L-: 5
Epoch: 6/500 Loss: 0.0251 LR: 1.0

KeyboardInterrupt: 

In [7]:
midi_model.save("test7.h5")

In [ ]:
for (line_nr, song) in read_songs(data_path):
    notes = []
    for bits in islice(midi_model.predict(song[:time_len]), 8*40):
        # print(bits_to_string(bits))
        # print (bits)
        notes.append(bits)
    create_and_save_midi_file(notes, "line_%d.mid" % line_nr)